In [4]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup

# Initialize the web driver
driver = webdriver.Edge()

# Define the URL of the TripAdvisor page
url = "https://www.tripadvisor.com/Hotel_Review-g304026-d312097-Reviews-Eko_Hotels_Suites-Lagos_Lagos_State.html"
driver.get(url)

# Get the page source
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Create a CSV file to write the data
csv_file = open("tripadvisor_reviews.csv", "w", newline='', encoding="utf-8")
csv_writer = csv.writer(csv_file)

# Function to extract review details
def extract_review_details(review_div):
    # Extract the review title
    review_title = review_div.find('div', class_='KgQgP MC _S b S6 H5 _a')
    title = review_title.text.strip() if review_title else None

    # Extract the review text
    review_text_div = review_div.find('span', class_='QewHA H4 _a')
    review_text = review_text_div.select_one(':first-child').text.strip() if review_text_div else None

    # Extract the stay date
    stay_date_span = review_div.find('span', class_='usajM')
    stay_date = stay_date_span.next_sibling.strip() if stay_date_span else None

    # Extract the trip type
    trip_type_span = review_div.find('span', class_='trip_type_label')
    trip_type = trip_type_span.next_sibling.strip() if trip_type_span else None

    # Extract the room tips
    room_tip_div = review_div.find('span', class_='tkWaG b')
    room_tip = room_tip_div.find_next_sibling('span').text if room_tip_div else None

    # Extract Review Date
    review_date_div = review_div.find('a', class_='ui_header_link uyyBf')
    author_info = review_date_div.next_sibling.strip() if review_date_div else None

    # Extract location of author
    location_div = review_div.find('span', class_='ui_icon map-pin-fill fXexN')
    author_location = location_div.next_sibling.strip() if location_div else None

    # Extract helpful votes, contributions, and review author
    author_div = review_div.find('div', class_='MziKN')
    elements = [span.text for span in author_div.select('span.phMBo > span')] if author_div else None

    # Extract the overall review rating
    rating_div = review_div.find('div', class_='Hlmiy F1')
    rating_span = rating_div.find('span', class_='ui_bubble_rating')
    rating_class = rating_span['class'][1].replace('bubble_', '') if rating_span else None

    # Extract the specific review rating
    rating_div = review_div.find_all('div', class_='hemdC S2 H2 WWOoy')
    specific_ratings = []
    for specific_rating in rating_div:
        value1 = specific_rating.find('span', class_='Nd').find_next_sibling().text
        value2 = specific_rating.find('span', class_='Nd').select_one(':first-child')['class'][1]
        specific_ratings.append(f"{value1} {value2}")

    # Create a dictionary to store all the extracted details
    review_details = {
        "Review Title": title,
        "Review Text": review_text,
        "Stay Date": stay_date,
        "Trip Type": trip_type,
        "Room Tips": room_tip,
        "Review Date": author_info,
        "Author Location": author_location,
        "Author Info": elements,
        "Overall Rating": rating_class,
        "Specific Ratings": specific_ratings,
    }

    # Create a list with the extracted details
    details_list = [
        title, review_text, stay_date, trip_type, room_tip,
        author_info, author_location, ', '.join(elements),
        rating_class, ', '.join(specific_ratings)
    ]

    # Write the details to the CSV file
    csv_writer.writerow(details_list)

    return review_details

# Find and extract review details
reviews = soup.find('div', class_='uNacK PS')
if reviews:
    review_divs = reviews.find_all('div', class_='YibKl MC R2 Gi z Z BB pBbQr')
    for count, review_div in enumerate(review_divs, 1):
        print('-----------')
        print(count)
        print('-----------')
        details = extract_review_details(review_div)
        for key, value in details.items():
            if value:
                print(f"{key}: {value}")


# Close the CSV file
csv_file.close()

# Close the web driver
driver.quit()


-----------
1
-----------
Review Title: AC
Review Text: Had difficulty adjusting the AC temperature. Could be that I did not understand how to operate it as it is a central HVAC system. I could have called the attention of personnel too, in hindsight and they might have provided clarification.
Stay Date: October 2023
Trip Type: Traveled solo
Review Date: wrote a review Oct 2023
Author Info: ['1 contribution']
Overall Rating: 50
Specific Ratings: ['Value bubble_50', 'Location bubble_50', 'Service bubble_50']
-----------
2
-----------
Review Title: Deterioration of service and quality
Review Text: For the past 5 years ive always stayed at the Eko main hotel. Ive noticed the deterioration of services and quality. More and more items are not available on the menus, even a simple pina colada from crossroads restaurant could not be made due to lack of ingredients. The room service menu is even worse- more than 50% of items are not available. Ive requested an iron to my room and had to wait 4